In [ ]:
def identity_block(X, f, filters, stage, block):
    """
    实现图3的恒等块

    参数：
        X - 输入的tensor类型的数据，维度为( m, 1001， 1)
        f - 整数，指定主路径中间的CONV窗口的维度
        filters - 整数列表，定义了主路径每层的卷积层的过滤器数量
        stage - 整数，根据每层的位置来命名每一层，与block参数一起使用。
        block - 字符串，据每层的位置来命名每一层，与stage参数一起使用。

    返回：
        X - 恒等块的输出，tensor类型，维度为(1001，1)

    """

    #定义命名规则
    conv_name_base = "res" + str(stage) + block + "_branch"
    bn_name_base   = "bn"  + str(stage) + block + "_branch"

    #获取过滤器
    F1, F2, F3 = filters

    #保存输入数据，将会用于为主路径添加捷径
    X_shortcut = X

    #主路径的第一部分
    ##卷积层
    X = Conv1D(filters=F1, kernel_size=1, strides=1 ,padding="valid",
               name=conv_name_base+"2a", kernel_initializer=glorot_uniform(seed=0))(X)
    ##归一化
    X = BatchNormalization(name=bn_name_base+"2a")(X)
    ##使用ReLU激活函数
    X = Activation("relu")(X)

    #主路径的第二部分
    ##卷积层
    X = Conv1D(filters=F2, kernel_size=f,strides=1, padding="same",
               name=conv_name_base+"2b", kernel_initializer=glorot_uniform(seed=0))(X)
    X = BatchNormalization(name=bn_name_base+"2b")(X)
    ##使用ReLU激活函数
    X = Activation("relu")(X)


    #主路径的第三部分
    ##卷积层
    X = Conv1D(filters=F3, kernel_size=1, strides=1, padding="valid",
               name=conv_name_base+"2c", kernel_initializer=glorot_uniform(seed=0))(X)
    X = BatchNormalization(name=bn_name_base+"2c")(X)
    ##没有ReLU激活函数

    #最后一步：
    ##将捷径与输入加在一起
    X = Add()([X,X_shortcut])
    ##使用ReLU激活函数
    X = Activation("relu")(X)

    return X

In [ ]:
def convolutional_block(X, f, filters, stage, block, s=2):
    #定义命名规则
    conv_name_base = "res" + str(stage) + block + "_branch"
    bn_name_base   = "bn"  + str(stage) + block + "_branch"

    #获取过滤器数量
    F1, F2, F3 = filters

    #保存输入数据
    X_shortcut = X
    
    #主路径
    ##主路径第一部分
    X = Conv1D(filters=F1, kernel_size=1, strides=s, padding="valid",
               name=conv_name_base+"2a", kernel_initializer=glorot_uniform(seed=0))(X)
    X = BatchNormalization(name=bn_name_base+"2a")(X)
    X = Activation("relu")(X)

    ##主路径第二部分
    X = Conv1D(filters=F2, kernel_size=f, strides=1, padding="same",
               name=conv_name_base+"2b", kernel_initializer=glorot_uniform(seed=0))(X)
    X = BatchNormalization(name=bn_name_base+"2b")(X)
    X = Activation("relu")(X)

#     ##主路径第三部分
#     X = Conv1D(filters=F3, kernel_size=1, strides=1, padding="valid",
#                name=conv_name_base+"2c", kernel_initializer=glorot_uniform(seed=0))(X)
#     X = BatchNormalization(name=bn_name_base+"2c")(X)

    #捷径
    X_shortcut = Conv1D(filters=F3, kernel_size=1, strides=s, padding="valid",
               name=conv_name_base+"1", kernel_initializer=glorot_uniform(seed=0))(X_shortcut)
    X_shortcut = BatchNormalization(name=bn_name_base+"1")(X_shortcut)

    #最后一步
    X = Add()([X,X_shortcut])
    X = Activation("relu")(X)

    return X

In [ ]:
# Construct ResNet50
def ResNet50(input_shape=(1001,1),classes=186):
    #定义tensor类型的输入数据
    X_input = Input(input_shape)

    #0填充
    X = keras.layers.convolutional.ZeroPadding1D(3)(X_input)

    #stage1
    X = Conv1D(filters=32, kernel_size=7, strides=2, name="conv1",
               kernel_initializer=glorot_uniform(seed=0))(X)
    X = BatchNormalization(name="bn_conv1")(X)
    X = Activation("relu")(X)
    X = MaxPooling1D(pool_size=3, strides=2)(X)

    #stage2
    X = convolutional_block(X, f=3, filters=[32,32,64], stage=2, block="a", s=1)
    X = identity_block(X, f=3, filters=[32,32,64], stage=2, block="b")
#     X = identity_block(X, f=3, filters=[64,64,256], stage=2, block="c")

    #stage3
    X = convolutional_block(X, f=3, filters=[64,64,128], stage=3, block="a", s=2)
    X = identity_block(X, f=3, filters=[64,64,128], stage=3, block="b")
#     X = identity_block(X, f=3, filters=[128,128,512], stage=3, block="c")
#     X = identity_block(X, f=3, filters=[128,128,512], stage=3, block="d")

    #stage4
    X = convolutional_block(X, f=3, filters=[128,128,256], stage=4, block="a", s=2)
    X = identity_block(X, f=3, filters=[128,128,256], stage=4, block="b")
#     X = identity_block(X, f=3, filters=[256,256,1024], stage=4, block="c")
#     X = identity_block(X, f=3, filters=[256,256,1024], stage=4, block="d")
#     X = identity_block(X, f=3, filters=[256,256,1024], stage=4, block="e")
#     X = identity_block(X, f=3, filters=[256,256,1024], stage=4, block="f")

    #stage5
    X = convolutional_block(X, f=3, filters=[128,128,256], stage=5, block="a", s=2)
    X = identity_block(X, f=3, filters=[128,128,256], stage=5, block="b")
#     X = identity_block(X, f=3, filters=[512,512,2048], stage=5, block="c")

    #均值池化层
    X = AveragePooling1D(pool_size=2,padding="same")(X)

    #输出层
    X = Flatten()(X)
    X = Dense(classes, activation="softmax", name="fc"+str(classes),
              kernel_initializer=glorot_uniform(seed=0))(X)


    #创建模型
    cnn_model = Model(inputs=X_input, outputs=X, name="ResNet50")

    return cnn_model